In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import copy
import random

2023-09-26 14:14:34.644713: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-26 14:14:35.115949: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
batch_size = 1024
learning_rate = 0.001

In [3]:
@tf.keras.saving.register_keras_serializable()
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(units=128, activation=tf.nn.leaky_relu)
        self.dense2 = tf.keras.layers.Dense(units=1024, activation=tf.nn.leaky_relu)
        self.dense3 = tf.keras.layers.Dense(units=128, activation=tf.nn.leaky_relu)
        self.dense4 = tf.keras.layers.Dense(units=1024, activation=tf.nn.leaky_relu)
        self.dense5 = tf.keras.layers.Dense(units=8)

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        output = self.dense5(x)
        return output

In [4]:
class ParaServer:
    def __init__(self):
        self.model = MLP()
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    def upload(self, grads):
        self.optimizer.apply_gradients(grads_and_vars=zip(grads, self.model.variables))
        return self.model
    def download(self):
        return self.model
    def initModel(self, x):
        self.model(x)

In [5]:
def valiAll(index_epoch):
    m = ps.download()
    model = copy.deepcopy(m)
    y_v_p = model(X_v)
    va_mse = tf.reduce_mean(tf.square(y_v_p - y_v))
    va_rmse = tf.sqrt(va_mse)
    va_mae = tf.reduce_mean(tf.abs(y_v_p - y_v))
    va_r2 = 1 - tf.reduce_sum(tf.square(y_v_p - y_v)) / tf.reduce_sum(tf.square(y_v - tf.reduce_mean(y_v)))
    print("mse:{} rmse:{} mae:{} r2:{}".format(va_mse, va_rmse, va_mae, va_r2))
    r2sv[index_epoch] = va_r2.numpy()

In [6]:
class Node:
    def __init__(self, id, freq):
        self.id = id
        self.freq = freq
        self.model = MLP()
        self.dataset1 = pd.read_csv('./20-24Trainset.csv', encoding='utf-8')
        self.dataset2 = pd.read_csv('./50-54Trainset.csv', encoding='utf-8')
        self.dataset = pd.concat([self.dataset1, self.dataset2], axis=0).sample(frac=1).reset_index(drop=True)
        self.dataset = self.dataset[self.dataset['freq'].isin(self.freq)]
        self.X = self.dataset.loc[:,'freq':'L2'].to_numpy(dtype = np.float32)
        self.y = self.dataset.loc[:,'S11r':'S41i'].to_numpy(dtype = np.float32)
        self.dataset_train = tf.data.Dataset.from_tensor_slices((self.X, self.y))
        self.dataset_train = self.dataset_train.shuffle(buffer_size=self.X.shape[0])
        self.dataset_train = self.dataset_train.batch(batch_size)
        self.dataset_train = self.dataset_train.prefetch(tf.data.experimental.AUTOTUNE)
    def train(self, index_epoch):
        self.model = ps.download()
        for X, y in self.dataset_train:
            with tf.GradientTape() as tape:
                y_pred = self.model(X)
                tr_mse = tf.reduce_mean(tf.square(y_pred - y))
            tr_rmse = tf.sqrt(tr_mse)
            tr_mae = tf.reduce_mean(tf.abs(y_pred - y))
            tr_r2 = 1 - tf.reduce_sum(tf.square(y_pred - y)) / tf.reduce_sum(tf.square(y - tf.reduce_mean(y)))
            grads = tape.gradient(tr_mse, self.model.variables)
            self.model = ps.upload(grads)
        # if epoch_index in np.arange(0, num_epochs, 25).tolist() or epoch_index == num_epochs - 1:
        print("node:{} epoch:{}".format(self.freq, index_epoch))
        print("train mse:{} rmse:{} mae:{} r2:{}".format(tr_mse, tr_rmse, tr_mae, tr_r2))
        r2s[self.id][index_epoch] = tr_r2.numpy()

In [7]:
r2s = {0:{}, 1:{}, 2:{}, 3:{}, 4:{}}
r2sv = {}

In [8]:
test_dataset = pd.read_csv("testset.csv", encoding='utf-8').sample(frac=1).reset_index(drop=True)
X_v = test_dataset.loc[:,'freq':'L2'].to_numpy(dtype = np.float32)
y_v = test_dataset.loc[:,'S11r':'S41i'].to_numpy(dtype = np.float32)

In [9]:
ps = ParaServer()
ps.initModel(X_v)

2023-09-26 14:14:36.709962: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9604 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5
2023-09-26 14:14:36.710503: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9621 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5


In [10]:
nodeList = [Node(0, [2.0, 5.0]), Node(1, [2.1, 5.1]), Node(2, [2.2, 5.2]), Node(3, [2.3, 5.3]), Node(4, [2.4, 5.4])]

In [11]:
orders = [0, 1, 2, 3, 4]
turn = [np.array([[92, 146], [158, 255], [347, 475], [531, 555]]), 
        np.array([[42, 116], [226, 277], [363, 423], [543, 600]]), 
        np.array([[0, 200], [214, 252], [271, 347], [474, 528]]),
        np.array([[68, 132], [173, 305], [418, 423], [563, 587]]),
        np.array([[7, 151], [216, 305], [357, 360], [420, 538]]),]
for i in range(600):
    random.shuffle(orders)
    for j in orders:
        for l, r in turn[j]:
            if l <= i < r:
                nodeList[j].train(i)
    valiAll(i)

2023-09-26 14:14:39.363225: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5627bdb7b120 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-26 14:14:39.363244: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2023-09-26 14:14:39.363249: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2023-09-26 14:14:39.366374: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-26 14:14:39.469076: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-09-26 14:14:39.567827: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


node:[2.2, 5.2] epoch:0
train mse:0.07567862421274185 rmse:0.27509748935699463 mae:0.21659544110298157 r2:0.3805719017982483
mse:0.07723215222358704 rmse:0.27790671586990356 mae:0.21820668876171112 r2:0.3674463629722595
node:[2.2, 5.2] epoch:1
train mse:0.0650981217622757 rmse:0.25514334440231323 mae:0.20155476033687592 r2:0.4667797088623047
mse:0.06700396537780762 rmse:0.25885123014450073 mae:0.20423826575279236 r2:0.4512181282043457
node:[2.2, 5.2] epoch:2
train mse:0.06107646971940994 rmse:0.24713653326034546 mae:0.19411788880825043 r2:0.4981561303138733
mse:0.05979503318667412 rmse:0.24453023076057434 mae:0.1920144110918045 r2:0.5102614164352417
node:[2.2, 5.2] epoch:3
train mse:0.051049571484327316 rmse:0.2259415239095688 mae:0.17743085324764252 r2:0.581964373588562
mse:0.05392992123961449 rmse:0.23222815990447998 mae:0.181097611784935 r2:0.5582983493804932
node:[2.2, 5.2] epoch:4
train mse:0.046764228492975235 rmse:0.21625038981437683 mae:0.170478954911232 r2:0.6167932152748108
m

In [24]:
for k, v in r2sv.items():
    print(v)

0.36744636
0.45121813
0.5102614
0.55829835
0.57672155
0.59171355
0.5803985
0.54091275
0.530311
0.553609
0.65252185
0.5131078
0.6726124
0.66876686
0.67864084
0.5032867
0.49833572
0.50099003
0.51613736
0.6901023
0.49104148
0.69060075
0.69635063
0.5013622
0.5163698
0.6956489
0.49803954
0.51043165
0.69561255
0.50274813
0.6984627
0.7070435
0.4921829
0.7084471
0.47449708
0.7075034
0.49061394
0.4888897
0.7029956
0.48546386
0.50566727
0.47770303
0.7114614
0.49067843
0.71195316
0.49941272
0.6519035
0.48103607
0.49381667
0.7151729
0.49254513
0.48092455
0.63849413
0.712142
0.63305616
0.7146537
0.63652194
0.64130914
0.48962045
0.71208966
0.63878137
0.47032058
0.6397207
0.64104253
0.7130977
0.6420907
0.7138994
0.48114717
0.71589833
0.654582
0.64051014
0.47666132
0.486534
0.6313716
0.6521373
0.65392905
0.4858315
0.6390941
0.6514478
0.7138605
0.65118587
0.47423923
0.6479465
0.6356795
0.64678985
0.7095667
0.6420176
0.71029496
0.64952815
0.471592
0.47955966
0.7134626
0.4344073
0.6493975
0.6488061
0.437